# Circulant Tensor Nuclear Norm Minimization

A special case of LCR-2D without regularization terms.

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox_2d(z, w, lmbda, denominator):
    N, T = z.shape
    temp1 = np.fft.fft2(lmbda * z - w) / denominator
    temp2 = 1 - N * T / (denominator * np.abs(temp1))
    temp2[temp2 <= 0] = 0
    return np.fft.ifft2(temp1 * temp2).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def flip_mat(X):
    temp = np.append(X, np.flip(X, axis = 1), axis = 1)
    return np.append(temp, np.flip(temp, axis = 0), axis = 0)

def inv_flip_mat(mat):
    dim = mat.shape
    N = int(dim[0] / 2)
    T = int(dim[1] / 2)
    temp = (mat[:, : T] + np.flip(mat[:, T :], axis = 1)) / 2
    return (temp[: N, :] + np.flip(temp[N :, :], axis = 0)) / 2

def LCR_2d(y_true, y, lmbda, gamma, tau, maxiter = 50):
    eta = 100 * lmbda
    if np.isnan(y).any() == False:
        pos_test = np.where((y_true != 0) & (y == 0))
    elif np.isnan(y).any() == True:
        pos_test = np.where((y_true > 0) & (np.isnan(y)))
        y[np.isnan(y)] = 0
    y_test = y_true[pos_test]
    flip_y_true = flip_mat(y_true)
    flip_y = flip_mat(y)
    N, T = flip_y.shape
    pos_train = np.where(flip_y != 0)
    y_train = flip_y[pos_train]
    z = flip_y.copy()
    w = flip_y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    ell_t = laplacian(T, tau)
    ell = np.fft.fft2(np.outer(ell_s, ell_t))
    denominator = lmbda + gamma * np.abs(ell) ** 2
    del y_true, y
    show_iter = 1
    for it in range(maxiter):
        x = prox_2d(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        y_hat = inv_flip_mat(x)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, y_hat[pos_test]))
            print(compute_rmse(y_test, y_hat[pos_test]))
            print()
    return y_hat

def LCR_3d(y_true, y, lmbda, gamma, tau_s, tau_t, maxiter = 50):
    dim1, dim2, dim3 = y.shape
    eta = 100 * lmbda
    if np.isnan(y).any() == False:
        pos_test = np.where((y_true != 0) & (y == 0))
    elif np.isnan(y).any() == True:
        pos_test = np.where((y_true > 0) & (np.isnan(y)))
        y[np.isnan(y)] = 0
    y_test = y_true[pos_test]
    flip_y_true = np.zeros((2 * dim1, 2 * dim2, dim3))
    flip_y = np.zeros((2 * dim1, 2 * dim2, dim3))
    for t in range(dim3):
        flip_y_true[:, :, t] = flip_mat(y_true[:, :, t])
        flip_y[:, :, t] = flip_mat(y[:, :, t])
    N, T, L = flip_y.shape
    pos_train = np.where(flip_y != 0)
    y_train = flip_y[pos_train]
    z = flip_y.copy()
    w = flip_y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    # ell_s = laplacian(N, tau_s)
    ell_t = laplacian(T, tau_t)
    ell = np.fft.fft2(np.outer(ell_s, ell_t))
    denominator = lmbda + gamma * np.abs(ell) ** 2
    del y_true, y
    show_iter = 10
    for it in range(maxiter):
        x = np.zeros((N, T, L))
        for t in range(dim3):
            x[:, :, t] = prox_2d(z[:, :, t], w[:, :, t], lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        y_hat = np.zeros((dim1, dim2, dim3))
        for t in range(dim3):
            y_hat[:, :, t] = inv_flip_mat(x[:, :, t])
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, y_hat[pos_test]))
            print(compute_rmse(y_test, y_hat[pos_test]))
            print()
    return y_hat

## HighD Dataset

Hyperparameter:

- $\lambda=10^{-3}NT$


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=3):
    fig = plt.figure(figsize = (2.5*2.5, 5))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower', 
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 5, vmax = 35, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/HighD/speed_matrix_full_46.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/HighD/speed_matrix_70_46.npy').transpose(1, 0, 2)

# for lane in range(3):
#     plot_speed_field(dense_tensor[:, :, lane],
#                      'speed_matrix_full_46_lane{}.png'.format(lane + 1))
#     plot_speed_field(sparse_tensor[:, :, lane],
#                      'speed_matrix_70_46_lane{}.png'.format(lane + 1))

import time
start = time.time()
N, T, L = sparse_tensor.shape
lmbda = 1e-3 * N * T
gamma = 0 * lmbda
tau_s = 1
tau_t = 2
maxiter = 100
tensor_hat = LCR_3d(dense_tensor, sparse_tensor, lmbda, gamma, tau_s, tau_t, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

# for lane in range(3):
#     plot_speed_field(tensor_hat[:, :, lane], 
#                      'HighD_speed_field_70_LCR_2D_rec_lane{}.png'.format(lane + 1))

<ipython-input-1-972d57671ea7>:20: RuntimeWarning: divide by zero encountered in true_divide
  temp2 = 1 - N * T / (denominator * np.abs(temp1))


10
0.6099151611688653
17.601873849451408

20
0.4093339780124503
12.818924372846341

30
0.2666694252380315
9.092467643817976

40
0.16987453743372286
6.226437584785453

50
0.10870867508974282
4.1384600598509635

60
0.0760804028014665
2.854571703180107

70
0.06276891034991629
2.2197220677508445

80
0.05762629895743574
1.9609680449896612

90
0.05615003096737754
1.887169653170882

100
0.05608405422960914
1.8679408560532873

Running time: 13 seconds.


## CitySim Dataset

Hyperparameter:

- $\lambda=10^{-4}NT$

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=5):
    fig = plt.figure(figsize = (2.3*2.3, 1.8))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower',
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 14, vmax = 35, fignum = 1, aspect='auto')
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/CitySim/speed_matrix_full.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/CitySim/speed_matrix_70_FB.npy').transpose(1, 0, 2)

# for lane in range(3):
#     plot_speed_field(dense_tensor[:, :, lane],
#                      'speed_matrix_full_lane{}.png'.format(lane + 1))
#     plot_speed_field(sparse_tensor[:, :, lane],
#                      'speed_matrix_70_lane{}.png'.format(lane + 1))

import time
start = time.time()
N, T, L = sparse_tensor.shape
lmbda = 1e-4 * N * T
gamma = 0 * lmbda
tau_s = 1
tau_t = 3
maxiter = 100
tensor_hat = LCR_3d(dense_tensor, sparse_tensor, lmbda, gamma, tau_s, tau_t, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

# for lane in range(3):
#     plot_speed_field(tensor_hat[:, :, lane], 
#                      'CitySim_speed_field_70_LCR_2D_rec_lane{}.png'.format(lane + 1))

<ipython-input-1-972d57671ea7>:20: RuntimeWarning: divide by zero encountered in true_divide
  temp2 = 1 - N * T / (denominator * np.abs(temp1))


10
0.12933736128839532
3.573515472925921

20
0.10223470162771996
2.9221988545765725

30
0.09743969790265358
2.804980125711941

40
0.09578189982041843
2.7722425458778854

50
0.09598923193140556
2.779449923940752

60
0.09499422877779558
2.7553810939155823

70
0.09416793676312299
2.735175987717144

80
0.09471057184116685
2.744753081407108

90
0.09472535491825756
2.74139645551234

100
0.09422923888382674
2.734053191242078

Running time: 6 seconds.


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>